# 문서 군집화(Clustering)와 문서간 유사도(Similarity) 측정

* 문서 군집화 : 동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류

* ref : https://techblog-history-younghunjo1.tistory.com/114

* BOW .Count Vectorizer : 단어 발생 빈도수에만 초점, 문서들간의 관계 파악 X
* Tf-idf 사용
* 벡터화 방법
    - BOW(Bag Of Words) : 단어 발생 빈도수에 기반, Feature 벡터화
    - Word embedding : 단어들간의 의미 관계(벡터들간의 방향) 고려해 벡터화

In [1]:
import pandas as pd
import glob, os
import matplotlib.pyplot as plt
import seaborn as sns

#-------------------- 주피터 셀 넓이 조절 ---------------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', None)

sns.set()

C:\Users\ASIA\AppData\Local\Temp\ipykernel_19508\2021393595.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# from sklearn.datasets import fetch_20newsgroups
# news_data = fetch_20newsgroups(subset='all',random_state=156)

In [2]:
path = './datasets/UCI_Opinion_Review'
all_files = glob.glob(os.path.join(path,"*.data"))
# all_files = os.listdir('./datasets/UCI_Opinion_Review')

filename_list = []
opinion_text = []
for file_ in all_files:
   #['', '/datasets/UCI_Opinion_Review\\accuracy_garmin_nuvi_255W_gps', 'txt', 'data']
    filename_ = file_.split('\\')[-1]  # 확장자 꺼내기
    filename = filename_.split('.')[0] # 파일이름 꺼내기
    filename_list.append(filename)
    
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    opinion_text.append(df.to_string())

document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head(1)

filename  \
0  accuracy_garmin_nuvi_255W_gps   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [4]:
# nltk.download('punkt')     # c:\ai\pythonproject\venv\nltk_data
# nltk.download('wordnet')
  

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
import string

print( string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [20]:
remove_punct_list=[]
for punct in string.punctuation:
    # print(ord(punct))
    remove_punct_list.append((ord(punct),None))
    remove_punct_dict=dict(remove_punct_list)

print(remove_punct_dict)

# remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
# print(remove_punct_dict)

{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [35]:
# 텍스트 단어들의 어근 원형을 추출하기 위해 함수 생성
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
import string

# ord('문자열') => 문자열의 ASCII코드를 반환
# dict(key, value)형태로 모든 구두점의 각 ASCII코드를 key값으로 넣어주자!
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]
def LemNormalize(text):
    #구두점 삭제해준 상태로 토큰화
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))  # translate : 딕셔너리 형태로 값을 전달하면 키를 값의 형태로 바꾸겠다는 것
                                                                                     #           : 각 텍스트에서 remove_punct_dict에 있는 특수문자가 있을 경우 해당 문자 없앰

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize,
                             stop_words='english', 
                             ngram_range=(1,2),  # 단어를 1개 혹은 연속된 2개의 단어의 형태로 잘라라
                             min_df=0.05, max_df=0.85)              # 빈도수 0.05이하, 0.85이상의 빈도수 단어들 제거
ftr_vect = tfidf_vect.fit_transform(document_df['opinion_text'])    # 벡터화
print(ftr_vect.shape)
print(ftr_vect.toarray())

(51, 4611)
[[0.04832472 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.01547441 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.0088014  0.         0.         ... 0.         0.         0.        ]]


In [48]:
vocab=tfidf_vect.vocabulary_
print(len(vocab.keys()))
# print(vocab)

4611


In [26]:
LemNormalize("I did not give it 5 stars is the fact that the speed limit it displays for the road you are on isn't 100% accurate")

['i',
 'did',
 'not',
 'give',
 'it',
 '5',
 'star',
 'is',
 'the',
 'fact',
 'that',
 'the',
 'speed',
 'limit',
 'it',
 'display',
 'for',
 'the',
 'road',
 'you',
 'are',
 'on',
 'isnt',
 '100',
 'accurate']

In [70]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, max_iter=10000, random_state=42)
cluster_label = kmeans.fit_predict(ftr_vect) # ftr_vect에서의 한 줄 == 하나의 문서 내에 있는 단어들의 중요도
                                             # (tfidf: 문서 내에서 특정 단어의 중요도)
document_df['cluster_label'] = cluster_label   # cluster_label 문서들이 분류된 군집 번호
# document_df.sort_values(by=['cluster_label']).head(2)
document_df['cluster_label'].value_counts()

2    25
1    16
0    10
Name: cluster_label, dtype: int64

In [60]:
# 문서의 feature(단어별) cluster_centers_
cluster_centers = kmeans.cluster_centers_
print(cluster_centers.shape)  #(클러스터 레이블, 벡터화feature(단어들)
print(cluster_centers)

(3, 4611)
[[0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]]


In [61]:
cluster_details = {}
cluster_details[0]={}
cluster_details[0]['cluster']=0
cluster_details

{0: {'cluster': 0}}

In [73]:
def get_cluster_details(cluster_model, cluster_data, feature_names,
                       cluster_num, top_n_features=10):
    cluster_details = {}
    center_feature_idx = cluster_model.cluster_centers_.argsort()[:,::-1]  #내림차순정렬
    print(f"cluster_centers index : {center_feature_idx}")
    
    for cluster_num in range(cluster_num):   # cluster_num=3 -> 0,1,2
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        top_ftr_idx = center_feature_idx[cluster_num, :top_n_features]   # 인덱스 top10
        print(f"인덱스 top10 : {top_ftr_idx}")
        top_ftr = [feature_names[idx] for idx in top_ftr_idx]            # 상위 10개 feature들의 center값들 반환
        print(f"상위 10개 feature의 center값 : {top_ftr}")
        top_ftr_val = cluster_model.cluster_centers_[cluster_num, top_ftr_idx].tolist()
        print(f"top_ftr_val : {top_ftr_val}")
        
        cluster_details[cluster_num]['top_features'] = top_ftr           # 개별 군집 정보 넣기
        cluster_details[cluster_num]['top_featrues_value'] = top_ftr_val
        filenames = cluster_data[cluster_data['cluster_label']==cluster_num]['filename']  ## 해당 cluster_num으로 분류된 파일명 넣기
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames    
    return cluster_details

def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print(f"#####Cluster Num: {cluster_num}")
        print()
        print("상위 10개 feature단어들:\n", cluster_detail['top_features'])
        print()
        print(f"Cluster {cluster_num}으로 분류된 문서들:\n{cluster_detail['filenames'][:5]}")
        print('-'*20)


In [63]:
feature_names = tfidf_vect.get_feature_names()  # tfidf_vect(ngram_range=(1,2))의 결과 : 1개 혹은 연속된 2개의 단어로 자른 형태
print(feature_names[:30])

['0 5', '0 great', '0 room', '00', '05', '06', '07', '08', '1 2', '1 block', '1 great', '10 did', '10 great', '10 inch', '10 min', '10 minute', '10 room', '100', '100 accurate', '1005ha', '101', '102', '103', '104', '105', '105 room', '106', '107', '108', '109']


In [74]:
cluster_details = get_cluster_details(cluster_model=kmeans, # Kmeans()  : 위에서 fit_predict으로 document_df대한 정보 가지고 있음 
                                     cluster_data=document_df,
                                     feature_names=feature_names,
                                     cluster_num=3,
                                     top_n_features=10)

cluster_centers index : [[2252 3612 2706 ... 2791 2789    0]
 [3466 2095 3648 ... 2272 2275    0]
 [3572  685 2342 ... 1750 3647 4610]]
인덱스 top10 : [2252 3612 2706 1069 1806 1807 4154  896 3022 3241]
상위 10개 feature의 center값 : ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
top_ftr_val : [0.2278046916838381, 0.1939512034790141, 0.17730089408820418, 0.12790439933840922, 0.12265896733770874, 0.1165812335567078, 0.10486543788164414, 0.1015256988633988, 0.09562358109597213, 0.09233847022967973]
인덱스 top10 : [3466 2095 3648 3871 1728 2517  678  988 3159 2984]
상위 10개 feature의 center값 : ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
top_ftr_val : [0.26083463293785525, 0.20029241494780795, 0.17546706020591651, 0.14901486167147948, 0.1277265040158775, 0.12576707418420113, 0.07042395128821588, 0.06971189120637236, 0.059533968934674365, 0.05842875064534637]
인덱스 top10 : [3572  68

In [66]:
print_cluster_details(cluster_details)

#####Cluster Num: 0

상위 10개 feature단어들:
 ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']

Cluster 0으로 분류된 문서들:
['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyota_camry_2007']
--------------------
#####Cluster Num: 1

상위 10개 feature단어들:
 ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']

Cluster 1으로 분류된 문서들:
['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo']
--------------------
#####Cluster Num: 2

상위 10개 feature단어들:
 ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']

Cluster 2으로 분류된 문서들:
['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_